# Project 2: CDI profitability analisys

Retrieving data from the Central Bank

## Steps:
1- Get input from the user and transform it </br>
2- Retrieve data from the Central Bank </br>
3- Calculate profitytability within the period of time

In [ ]:
%pip install python-bcb

In [1]:
from datetime import datetime, date
from matplotlib import pyplot as plt
import numpy as np
from bcb import sgs

# 1- Get input from the user and transform it

In [3]:
capital = float(input("Please, inform the initial investment: "))
frequency = input("Please, inform the frequency (Y, M, D): ")
start = input("Please, inform the initial date later than 1995/01/01 (YYYY/MM/DD): ")
end = input("Please, inform the final date (YYYY/MM/DD): ")

In [5]:
initial_date = datetime.strptime(start, "%Y/%m/%d")
final_date = datetime.strptime(end, "%Y/%m/%d")

# 2- Retrieve SELIC Data from Central Bank 

In [7]:
selic = sgs.get({"selic":11}, start = initial_date, end = final_date)
selic = selic/100
selic.head()


,selic
Date,
1996-01-02,0.001200
1996-01-03,0.001197
1996-01-04,0.001147
1996-01-05,0.001137
1996-01-08,0.001137


# 3- Calculating profitytability

In [8]:
accumulated_capital = capital * (1 + selic["selic"]).cumprod() - 1
accumulated_capital

Date
1996-01-02     2001.400000
1996-01-03     2003.796212
1996-01-04     2006.095052
1996-01-05     2008.376456
1996-01-08     2010.660454
                  ...     
2023-12-22    79778.423782
2023-12-26    79813.318504
2023-12-27    79848.228489
2023-12-28    79883.153743
2023-12-29    79918.094273
Name: selic, Length: 7029, dtype: float64

In [12]:
frequency_capital = accumulated_capital.resample(frequency).last()
frequency_capital = frequency_capital.dropna()
frequency_capital


/tmp/ipykernel_191083/3631833244.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  frequency_capital = accumulated_capital.resample(frequency).last()


Date
1996-01-31     2050.529931
1996-02-29     2098.770944
1996-03-31     2145.403115
1996-04-30     2189.791724
1996-05-31     2233.886953
                  ...     
2023-08-31    76966.473078
2023-09-30    77715.290974
2023-10-31    78490.563220
2023-11-30    79209.536383
2023-12-31    79918.094273
Freq: ME, Name: selic, Length: 336, dtype: float64

# Finding the best day to invest in a 500 days window between 1/1/2000 and 31/3/2022

In [13]:
initial_date_2 = date(2000, 1, 1)
final_date_2 = date(2022, 3, 31)

selic_2 = sgs.get({"selic": 11}, start = initial_date_2, end = final_date_2)/100
selic_2

,selic
Date,
2000-01-03,0.000692
2000-01-04,0.000692
2000-01-05,0.000692
2000-01-06,0.000693
2000-01-07,0.000693
...,...
2022-03-25,0.000437
2022-03-28,0.000437
2022-03-29,0.000437


In [15]:
#calculating the window's profitabylity
window_500 = ((1+ selic_2).rolling(window = 500).apply(np.prod) - 1)
window_500

,selic
Date,
2000-01-03,NaN
2000-01-04,NaN
2000-01-05,NaN
2000-01-06,NaN
2000-01-07,NaN
...,...
2022-03-25,0.086543
2022-03-28,0.086864
2022-03-29,0.087185


In [16]:
window_500 = window_500.reset_index()
window_500["initial date"] = window_500["Date"].shift(500)
window_500

,Date,selic,initial date
0,2000-01-03,NaN,NaT
1,2000-01-04,NaN,NaT
2,2000-01-05,NaN,NaT
3,2000-01-06,NaN,NaT
4,2000-01-07,NaN,NaT
...,...,...,...
5583,2022-03-25,0.086543,2020-03-27
5584,2022-03-28,0.086864,2020-03-30
5585,2022-03-29,0.087185,2020-03-31
5586,2022-03-30,0.087506,2020-04-01


In [20]:
window_500 = window_500.dropna()
window_500.columns = ["Final Date", "SELIC", "Initial Date"]
window_500

,Final Date,SELIC,Initial Date
500,2002-01-02,0.377688,2000-01-03
501,2002-01-03,0.377689,2000-01-04
502,2002-01-04,0.377689,2000-01-05
503,2002-01-07,0.377688,2000-01-06
504,2002-01-08,0.377687,2000-01-07
...,...,...,...
5583,2022-03-25,0.086543,2020-03-27
5584,2022-03-28,0.086864,2020-03-30
5585,2022-03-29,0.087185,2020-03-31
5586,2022-03-30,0.087506,2020-04-01


In [21]:
max_profit = window_500["SELIC"].max()
best_window = window_500[window_500["SELIC"] == max_profit]
best_window

,Final Date,SELIC,Initial Date
956,2003-10-22,0.466967,2001-10-26
